<a href="https://colab.research.google.com/github/profitakeo/Real-Time-Measurement-of-Local-Traffic-Using-Autonomous-Vehicle-Open-Datasets/blob/master/Traffic_Calculation_(Waymo_Open_Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Measuring Traffic using Waymo Open Dataset

- Dataset available at https://waymo.com/open/download/
- Run using Google Colab: https://colab.research.google.com
- Restart runtime before running
- Based on [Waymo Open Dataset Tutorial](https://colab.research.google.com/github/waymo-research/waymo-open-dataset/blob/master/tutorial/tutorial.ipynb)
- Found in [GitHub repository](https://github.com/profitakeo/Real-Time-Measurement-of-Local-Traffic-Using-Autonomous-Vehicle-Open-Datasets)




## Install waymo_open_dataset package

In [ ]:
!rm -rf waymo-od > /dev/null
!git clone https://github.com/waymo-research/waymo-open-dataset.git waymo-od
!cd waymo-od && git branch -a
!cd waymo-od && git checkout remotes/origin/r1.0
!pip3 install --upgrade pip

Cloning into 'waymo-od'...
remote: Enumerating objects: 884, done.
remote: Total 884 (delta 0), reused 0 (delta 0), pack-reused 884
Receiving objects: 100% (884/884), 14.19 MiB | 5.35 MiB/s, done.
Resolving deltas: 100% (580/580), done.
* master
  remotes/origin/HEAD -> origin/master
  remotes/origin/master
  remotes/origin/r1.0
  remotes/origin/r1.0-tf1.15
  remotes/origin/r1.0-tf2.0
  remotes/origin/r1.2
Note: checking out 'remotes/origin/r1.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at a66eb0a Merge branch 'master' into r1.0
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.4)


## Import libraries

In [ ]:
%tensorflow_version 1.x # Change to tensorflow ver. 1
import tensorflow as tf 
print(tf.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.x # Change to tensorflow ver. 1`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
1.15.2


In [ ]:
!pip3 install waymo-open-dataset
import os
import math
import numpy as np
import pandas as pd
import itertools
import tensorflow as tf

tf.enable_eager_execution()

from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

## Mount Google Drive locally

In [ ]:
from google.colab import drive, files
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## Functions

### Showing camera image for a frame

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0}) # Ignore warning about 20+ figures consuming memory
import matplotlib.patches as patches

def show_camera_image(camera_image, camera_labels, layout, cmap=None):
  """Show a camera image and the given camera labels"""

  ax = plt.subplot(*layout)

  # Draw the camera labels
  for camera_labels in frame.camera_labels:
    # Ignore camera labels that do not correspond to this camera
    if camera_labels.name != camera_image.name:
      continue

    # Iterate over the individual labels
    for label in camera_labels.labels:
      # Draw the object bounding box
      ax.add_patch(patches.Rectangle(
        xy=(label.box.center_x - 0.5 * label.box.length,
            label.box.center_y - 0.5 * label.box.width),
        width=label.box.length,
        height=label.box.width,
        linewidth=1,
        edgecolor='red',
        facecolor='none'))

  # Show/download camera image
  plt.imshow(tf.image.decode_jpeg(camera_image.image), cmap=cmap)
  plt.title(open_dataset.CameraName.Name.Name(camera_image.name))
  plt.grid(False)
  plt.axis('off')
  frame_img = 'frame' + str(i) + '.png'
  plt.savefig('/content/Extracted_Data/' + frame_img)

### Convert frame data to DataFrame

In [ ]:
import nltk
nltk.download('punkt')
import math

def frame_to_dataframe(frame_data, frame_num):
  """Convert frame data to DataFrame"""

  nltk_tokens = nltk.word_tokenize(str(frame_data)) # Tokenize frame data
  df = pd.DataFrame()

  # Search frame data
  for i in range(len(nltk_tokens)):
    # Append vehicle data obtained from laser
    if nltk_tokens[i] == 'laser_labels':
      laser_row = [[ nltk_tokens[i], nltk_tokens[i+47], nltk_tokens[i+6], nltk_tokens[i+9], nltk_tokens[i+12], \
                nltk_tokens[i+15], nltk_tokens[i+18], nltk_tokens[i+21], nltk_tokens[i+24], nltk_tokens[i+30],\
                nltk_tokens[i+33], nltk_tokens[i+36], nltk_tokens[i+39], nltk_tokens[i+43] ]]
      df = df.append(laser_row, ignore_index=True)

    # Append vehicle data obtained from camera
    if nltk_tokens[i] == 'camera_labels':
      if nltk_tokens[i+5] == 'labels':
        camera_row = [[ nltk_tokens[i]+nltk_tokens[i+4], nltk_tokens[i+28], nltk_tokens[i+11], nltk_tokens[i+14],\
                    'NaN', nltk_tokens[i+17], nltk_tokens[i+20], 'NaN', 'NaN', 'NaN', 'NaN','NaN', 'NaN',\
                    nltk_tokens[i+24] ]]
        df = df.append(camera_row, ignore_index=True)
      elif nltk_tokens[i+5] == '}':
        continue
      for m in range(1, math.floor((len(nltk_tokens)-i)/31)):
        if nltk_tokens[i+5+26*m] != 'labels':
          break
        elif nltk_tokens[i+5+26*m] == 'labels':
          j = (i+5+26*m)
          values = [[ nltk_tokens[i]+nltk_tokens[i+4], nltk_tokens[j+23], nltk_tokens[j+6], nltk_tokens[j+9],\
                'NaN', nltk_tokens[j+12], nltk_tokens[j+15], 'NaN', 'NaN', 'NaN', 'NaN','NaN', 'NaN',\
                nltk_tokens[j+19] ]];
          df = df.append(values, ignore_index=True)
  
  # Formatting DataFrame
  df.columns = ['device','iD', 'center_x', 'center_y', 'center_z', 'width', 'length', 'height', \
                'heading', 'speed_x', 'speed_y', 'accel_x', 'accel_y', 'type'];
  df[['center_x', 'center_y', 'center_z', 'width', 'length', 'height', 'heading', 'speed_x', 'speed_y', 'accel_x', 'accel_y']] = \
   df[['center_x', 'center_y', 'center_z', 'width', 'length', 'height', 'heading', 'speed_x', 'speed_y', 'accel_x', 'accel_y']].astype('float64');
  
  # Convert DataFrame to .xslx
  frame_xlsx ='frame' + str(frame_num) + '.xlsx'
  df.to_excel(r'/content/Extracted_Data/' + frame_xlsx)
  
  return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Calculating AV speed



In [ ]:
def find_AV_speed(df1, df2, frame_num):
  """Calculating AV speed"""

  # Determine relative displacement of stationary sign object between 2 frames
  is_sign = df1['type']=='TYPE_SIGN' # Filter signs from DF1
  df_sign = df1[is_sign]
  for i in range(len(df_sign)): # Search all signs in Frame 1
    AV_dist_x_1 = df_sign['center_x'].iloc[i] # Find location of sign in Frame 1
    AV_dist_y_1 = df_sign['center_y'].iloc[i]
    is_sign_2 = df2['iD'] == df_sign.iloc[i]["iD"] # Go to DF2 and identify same sign
    df_sign_2 = df2[is_sign_2]
    if is_sign_2.any() == True: # Stop search if sign is also found in Frame 2
      break
  if df_sign_2.empty == True:
    print('No signs in 2 consecutive frames. Calculation can no longer be performed!')
  AV_dist_x_2 = df_sign_2['center_x'].iloc[0] # Obtain location of sign in Frame 2 
  AV_dist_y_2 = df_sign_2['center_y'].iloc[0]
 
  # Find speed of AV (m/s)
  frame_rate = 10 # Hertz; given by Waymo Open Dataset
  frame_len = 1/frame_rate
  AV_speed_x =  -(AV_dist_x_2 - AV_dist_x_1) / frame_len
  AV_speed_y =  -(AV_dist_y_2 - AV_dist_y_1) / frame_len

  print(f'Frame {frame_num-1}: Sign x-coordinate = {AV_dist_x_1:.3f} m,\
 Sign y-coordinate = {AV_dist_y_1:.3f} m\n\
 Frame {frame_num}: Sign x-coordinate = {AV_dist_x_2:.3f} m,\
 Sign y-coordinate = {AV_dist_y_2:.3f} m,\n\
 AV x-speed = {AV_speed_x:.3f} m/s,\
 AV y-speed = {AV_speed_y:.3f} m/s')
  #print( round(AV_dist_x_1, 3), round(AV_dist_y_1,3), round(AV_dist_x_2, 3), round(AV_dist_y_2, 3),
  #        round(AV_speed_x, 3), round(AV_speed_y, 3) )

  # Find x-distance travelled by AV (m)
  AV_dist_x = -(AV_dist_x_2 - AV_dist_x_1)

  return AV_speed_x, AV_speed_y, AV_dist_x

### Calculating traffic

In [ ]:
def calc_traffic(df1, df2, empty_DF, AV_cum_dist, AV_speed_x, AV_speed_y, AV_dist_x, frame_num):
  """Calculating traffic"""

  # Filtering DataFrame before calculating
  AV_len = 5.17 # metres; Waymo vehicle length
  df_AV = {'device': 'laser_labels', 'iD': 'AV', 'center_x':0, 'center_y':0, 'center_z':0, \
           'width':np.NaN, 'length':AV_len, 'height':np.NaN, 'heading':0, 'speed_x':AV_speed_x,\
           'speed_y':AV_speed_y, 'accel_x':np.NaN, 'accel_y':np.NaN, 'type': 'TYPE_VEHICLE'} 
  df2 = df2.append(df_AV, ignore_index=True) # Append AV row data to DF2

  is_veh = df2['type']=='TYPE_VEHICLE' # Show vehicle objects ...
  df_veh = df2[is_veh]

  is_laser = df_veh['device']=='laser_labels' # ... captured by LiDAR/laser ...
  df_laser = df_veh[is_laser] 

  df_ord = df_laser.sort_values(by=['center_x']) # ... in ascending center_x values ...

  road_width = 3.7 # metres; standard lane width given by U.S. Interstate Highway standards
  in_lane = abs(df_ord['center_y']) <= road_width/2 # ... travelling in the same straight lane as the AV ...
  df_lane = df_ord[in_lane]

  heading_max = math.pi/4
  fwd_dir = abs(df_lane['heading']) <= heading_max # ... and heading < Pi/4 rad.
  df_fwd = df_lane[fwd_dir]

  # Cumulative x-distance (metres) travelled by AV
  cum_dist = {'AV Cumulative Distance (m)': [AV_dist_x]}
  cum_dist_df = pd.DataFrame(cum_dist, columns = ['AV Cumulative Distance (m)'])
  AV_cum_dist = AV_cum_dist.append(cum_dist_df) 

  # Calculating vehicle inputs
  input = []  
  for index, row in enumerate(df_fwd.itertuples()):
    x_b = row.center_x - row.length*0.5*math.cos(row.heading) # Rear bumper x-coordinate
    y_b = row.center_y - row.length*0.5*math.sin(row.heading) # Rear bumper y-coordinate
    x_f = row.center_x + row.length*0.5*math.cos(row.heading) # Front bumper x-coordinate
    y_f = row.center_y + row.length*0.5*math.sin(row.heading) # Front bumper y-coordinate
    iD = row.iD
    speed_tot = math.sqrt(row.speed_x**2+row.speed_y**2) # Defining total speed
    center_x = row.center_x
    length = row.length
    input.append([iD, speed_tot, center_x, length, x_b, y_b, x_f, y_f])
  df_0 = pd.DataFrame(input, columns=['iD', 'speed_tot', 'center_x', 'length','x_b', 'y_b', 'x_f', 'y_f'])
  is_mov = abs(df_0['speed_tot']) > 0 # Filtering out stationary/parked vehicles defined by speed = 0 m/s
  df_0 = df_0[is_mov]

  # Shift rows for calculation between consecutive rows
  # Source: https://stackoverflow.com/questions/42664418/referencing-to-the-next-index-in-iterrows
  df_0['shifted_x_b'] = df_0['x_b'].shift(-1)
  df_0['shifted_y_b'] = df_0['y_b'].shift(-1)
  df_0['shifted_x_f'] = df_0['x_f'].shift(-1)
  df_0['shifted_y_f'] = df_0['y_f'].shift(-1)

  # Calculating more vehicle inputs
  df_1 = []
  for index, row in enumerate(df_0.itertuples()):
    s = math.sqrt((row.shifted_x_b - row.x_f) **2 + (row.shifted_y_b - row.y_f) **2) # Spacing (front to rear bumper)
    f = math.sqrt((row.shifted_x_f - row.x_f) **2 + (row.shifted_y_f - row.y_f) **2) # Spacing (front to front bumper)
    h = f / row.speed_tot # Headway
    df_1.append([s,f,h])
  df_1 = pd.DataFrame(df_1, columns=['s', 'f', 'h'])
  df_2 = pd.concat([df_0, df_1], axis=1)
  df_3 = df_2.drop(columns=['shifted_x_b', 'shifted_y_b', 'shifted_x_f', 'shifted_y_f']) # Removed shifted columns

  # Calculating traffic metrics
  if df_3['h'].sum() == 0:
    q_flow = ''
  elif df_3['h'].sum() != 0:
    q_flow = len(df_3) / df_3['h'].sum() * 3600 # veh/hr/lane
  if df_3['s'].sum() == 0:
    k_density = ''
  elif df_3['s'].sum() != 0:
    k_density = len(df_3) / df_3['s'].sum() * 1000 # veh/km/lane
  v_avgspeed = df_3['speed_tot'].mean() * 3.6 # km/h
  d_framelenx = (df_3['center_x'].max() + df_3['length'][df_3['center_x'].idxmax()]*0.5)- \
    (df_3['center_x'].min() - df_3['length'][df_3['center_x'].idxmin()]*0.5) # Distance between first/last vehicle
  
  frame_rate = 10 # Hertz
  final_data = {'q (veh/h)': [q_flow], \
                'k (veh/km)': [k_density], \
                'v (km/h)':[v_avgspeed], \
                'd (Analysis Distance in m)': [d_framelenx], \
                'x (AV Distance in m)': [AV_dist_x], \
                'Frame': [frame_num], \
                'Time (s)': [(frame_num-1)/frame_rate], \
                'AV Cumulative Distance (m)': [AV_cum_dist.sum().sum()], \
                'Predicted q': [k_density*v_avgspeed], \
                'Actual vs. Predicted q (% difference)': [(k_density*v_avgspeed-q_flow)/q_flow*100]}
  final_data = pd.DataFrame(final_data, columns = ['q (veh/h)', \
                                                   'k (veh/km)', \
                                                   'v (km/h)', \
                                                   'd (Analysis Distance in m)', \
                                                   'x (AV Distance in m)', \
                                                   'Frame', \
                                                   'Time (s)', \
                                                   'AV Cumulative Distance (m)', \
                                                   'Predicted q', \
                                                   'Actual vs. Predicted q (% difference)'])
  final_data = empty_DF.append(final_data, ignore_index = True)
  return final_data, AV_cum_dist

## Calculate traffic for all frames

Before running:
- Download .tfrecord files from [Waymo Open Dataset](https://waymo.com/open/download/) & upload to Google Drive
- Create new folder 'Extracted_Data' under /content
- Ensure file name and start/end frame are set correctly
- To avoid inactivity, open console (ctrl+shift+i OR command+option+i) and run [function](https://medium.com/p/717b88a128c0/responses/show) shown below.



In [ ]:
#function ConnectButton(){
#    console.log("Connect pushed")
#    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
#}
#setInterval(ConnectButton,60000)

In [ ]:
# Parameters
j = 1 # Start frame; min = 1
k = 200 # End frame; max = 200
FILENAME = '/gdrive/My Drive/Colab Notebooks/training_0002.tar/segment-11392401368700458296_1086_429_1106_429_with_camera_labels.tfrecord'

# Initialising
i = 0 # Frame count; initialise as 0
i2 = 1 # Frame pair count; initialise as 1
final_data_append = pd.DataFrame()
AV_cum_dist_append = pd.DataFrame()

dataset = tf.data.TFRecordDataset(FILENAME, compression_type='')
for data in dataset:

  i += 1 # Count number of frames

  if i<=j-1:
    continue
  frame = open_dataset.Frame()
  frame.ParseFromString(bytearray(data.numpy()))
  plt.figure(figsize=(25, 20))

  for index, image in enumerate(frame.images):
    show_camera_image(image, frame.camera_labels, [3, 3, index+1])
    if i2%2 != 0: # For 1st frame in pair (where index is odd), generate DF
      df1 = frame_to_dataframe(frame, i)
    elif i2%2 == 0: # For 2nd frame in pair (where index is even), generate DF
      df2 = frame_to_dataframe(frame, i)
      if (index % 5) == 0:
        AV_speed_x, AV_speed_y, AV_dist_x = find_AV_speed(df1, df2, i)
        final_data_append, AV_cum_dist_append = calc_traffic(df1, df2, final_data_append, \
                                         AV_cum_dist_append, AV_speed_x, \
                                         AV_speed_y, AV_dist_x, i)
        final_data_xlsx ='final_data' + str(i) + '.xlsx'
        final_data_append.to_excel(r'/content/Extracted_Data/' + final_data_xlsx, sheet_name='Raw')
      else:
        pass
      df1=df2 # Assign 1st frame as 2nd frame
      i2 += 1
 
  i2 += 1

  if i==k: 
    break

# Download zip folder
!zip -r /content/Extracted_Data/Extracted_Data.zip /content/Extracted_Data
from google.colab import files
files.download('/content/Extracted_Data/Extracted_Data.zip')

## Obtaining DataFrame for Birds Eye Visualisation

Before running:
- Ensure file name and start/end frame are set correctly
- Upload files "framei.xlsx" (for i = 1, ... ,200), which were created from previous step, to relevant Google Drive folder

In [ ]:
# Parameters
j = 1 # Start frame; min = 1
k = 200 # End frame; max = 200

# Initialising
birds_eye = pd.DataFrame() 

for y in range(k-j+1):
  FILENAME = '/gdrive/My Drive/Colab Notebooks/training_0002.tar/Frame Data (0002_11392) OK/frame' + str(y+1) + '.xlsx'
  df = pd.read_excel(FILENAME)
  df[['center_x', 'center_y', 'center_z', 'width', 'length', 'height', 'heading', 'speed_x', 'speed_y', 'accel_x', 'accel_y']]= \
    df[['center_x', 'center_y', 'center_z', 'width', 'length', 'height', 'heading', 'speed_x', 'speed_y', 'accel_x', 'accel_y']].astype('float64')
  df = df.drop(df.columns[[0]],axis=1)

  AV_len = 5.17 # metres; Waymo vehicle length
  df_AV = {'device': 'laser_labels', 'iD': 'AV', 'center_x':0, 'center_y':0, \
           'center_z':0, 'width':np.NaN, 'length':AV_len, 'height':np.NaN, \
           'heading':0, 'speed_x':0, 'speed_y':0, 'accel_x':np.NaN, \
           'accel_y':np.NaN, 'type': 'TYPE_AV'} 
  df = df.append(df_AV, ignore_index=True) # Append AV row data to DF2

  is_veh = df['device']=='laser_labels' # Filter for vehicles captured by LiDAR
  df_veh = df[is_veh]
 
  # Populating DataFrame
  df_0 = [] 
  for index, row in enumerate(df_veh.itertuples()):
    frame_num = y+1
    iD = row.iD
    center_x = row.center_x
    center_y = row.center_y
    heading = row.heading
    length = row.length
    type = row.type
    df_0.append([frame_num, iD, center_x, center_y,  heading, length, type])
  df_1 = pd.DataFrame(df_0, columns=['Frame','iD', 'center_x', 'center_y', 'heading', 'length', 'type'])
  birds_eye = birds_eye.append(df_1, ignore_index=True)

# Download zip folder
birds_eye.to_excel(r'/content/Extracted_Data/' + 'birds_eye_data (complete).xlsx', sheet_name='Raw')
!zip -r /content/Extracted_Data/Extracted_Data.zip /content/Extracted_Data
files.download('/content/Extracted_Data/Extracted_Data.zip')

  adding: content/Extracted_Data/ (stored 0%)
  adding: content/Extracted_Data/birds_eye_data (complete).xlsx (deflated 4%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

After running the last 2 steps, you should now have the following 2 files:
- final_data200.xlsx
-  birds_eye_data (complete).xlsx

Use as input for the .R files in source [GitHub repository](https://github.com/profitakeo/Real-Time-Measurement-of-Local-Traffic-Using-Autonomous-Vehicle-Open-Datasets).

